# Pivote

## Extracción de links

Todo

In [5]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime
from selenium.webdriver.chrome.options import Options

def create_driver(headless=True):
    """Crea y retorna un WebDriver con la opción de ser headless si se desea."""
    chrome_options = Options()
    if headless:
        chrome_options.add_argument("--headless")  # Activar modo headless
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def extract_articles_from_url2(driver, url, existing_df=None):
    """
    Extrae información de artículos desde una URL y agrega los resultados a un DataFrame existente.
    
    Args:
        driver (WebDriver): El controlador de Selenium para navegar por la página.
        url (str): URL de la página para extraer artículos.
        existing_df (pd.DataFrame, optional): DataFrame al que agregar los artículos extraídos. Si no se proporciona, se crea uno nuevo.
    
    Returns:
        pd.DataFrame: DataFrame con los artículos extraídos agregados al existente (o uno nuevo si no se proporciona).
    """
    try:
        driver.get(url)
        # Esperar a que cargue el contenido
        time.sleep(3)

        # Obtener todos los artículos dentro de las etiquetas <article>
        articles = driver.find_elements(By.CSS_SELECTOR, "div.section-archive__list__grid__item")

        data = []
        for article in articles:
            # Imagen
            try:
                img = article.find_element(By.CSS_SELECTOR, "img.wp-post-image")
                image_url = img.get_attribute("src")
            except:
                image_url = None

            # Título y enlace
            try:
                title_element = article.find_element(By.CSS_SELECTOR, "h3")
                title = title_element.text.strip()
                link = article.find_element(By.CSS_SELECTOR, "a.page-tile__enlace").get_attribute("href")
            except:
                continue  # Si no hay título o enlace, saltamos este artículo

            # Fecha
            try:
                date = article.find_element(By.CSS_SELECTOR, "time.updated").text.strip()
            except:
                date = "No especificada"

            # Autor
            try:
                author = article.find_element(By.CSS_SELECTOR, "span.autor").text.strip()
            except:
                author = "No especificado"

            # Contenido
            try:
                content_element = article.find_element(By.CSS_SELECTOR, "p.entry-terms")
                content = content_element.text.strip()
            except:
                content = None

            # Crear diccionario con los datos
            article_data = {
                'Título': title,
                'Enlace': link,
                'Fecha Publicación': date,
                'Autor': author,
                'Contenido': content,
                'URL Imagen': image_url,
                'Fecha Consulta': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }

            data.append(article_data)

        # Crear DataFrame con los nuevos datos
        new_df = pd.DataFrame(data)

        # Si ya existe un DataFrame proporcionado, concatenar los nuevos datos al existente
        if existing_df is not None:
            existing_df = pd.concat([existing_df, new_df], ignore_index=True)
        else:
            # Si no existe, devolver el nuevo DataFrame
            existing_df = new_df

        return existing_df

    except Exception as e:
        print(f"Error en la extracción: {str(e)}")
        return existing_df if existing_df is not None else pd.DataFrame()

def extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=100, headless=True):
    """
    Itera sobre varias páginas y extrae los artículos, deteniéndose cuando una página no se encuentra.
    
    Args:
        base_url (str): URL base de las páginas, debe contener un marcador de página, e.g., 'page/{page_number}/'.
        start_page (int): Número de página inicial para empezar a iterar (por defecto es la página 1).
        max_pages (int): Número máximo de páginas a intentar antes de detener la búsqueda (por defecto es 100).
        headless (bool): Si se debe ejecutar el navegador en modo headless (sin abrir ventana). Por defecto es True.
    
    Returns:
        pd.DataFrame: DataFrame con los artículos extraídos de todas las páginas procesadas.
    """
    driver = create_driver(headless)  # Inicializar el WebDriver solo una vez
    current_page = start_page
    all_articles_df = pd.DataFrame()  # Inicializamos un DataFrame vacío

    while current_page <= max_pages:
        # Construir la URL de la página actual
        #url = f"{base_url}{current_page}/"  # Aquí el formato de la URL, el número de página se añade al final
        url = f"{base_url}" 
        print(f"Extrayendo artículos de: {url}")

        # Llamar a la función extract_articles_from_url2 para obtener artículos de la página
        all_articles_df = extract_articles_from_url2(driver, url, all_articles_df)

        # Verificar si se encontró una página de error (por ejemplo, página no encontrada)
        if all_articles_df is None or len(all_articles_df) == 0:
            print(f"No se encontraron artículos en la página {current_page}, o la página no existe.")
            break

        current_page += 1  # Pasar a la siguiente página

    driver.quit()  # Cerrar el WebDriver después de todas las iteraciones
    return all_articles_df


Contenidos (Todos)

In [2]:
base_url = "https://www.pivotes.cl/contenidos/#todos"
contenidos = extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=1, headless=False)

Extrayendo artículos de: https://www.pivotes.cl/contenidos/#todos1/


In [3]:
contenidos

,Título,Enlace,Fecha Publicación,Autor,Contenido,URL Imagen,Fecha Consulta
0,Sistema político que hay que cuidar,https://www.pivotes.cl/contenidos/cartas-al-di...,"29 noviembre, 2024",Por: José Antonio Valenzuela,CARTAS AL DIRECTOR,None,2024-11-29 12:04:57
1,Si yo fuera Donald Trump,https://www.pivotes.cl/contenidos/columnas-de-...,"28 noviembre, 2024",Por: José Antonio Valenzuela,COLUMNAS DE OPINIÓN,https://www.pivotes.cl/web/wp-content/uploads/...,2024-11-29 12:04:57
2,Puntos de inflexión para el empleo público,https://www.pivotes.cl/contenidos/columnas-de-...,"28 noviembre, 2024",Por: Rafael Palacios,COLUMNAS DE OPINIÓN,https://www.pivotes.cl/web/wp-content/uploads/...,2024-11-29 12:04:57
3,Ley Lafkenche: No existen las balas de plata,https://www.pivotes.cl/contenidos/columnas-de-...,"28 noviembre, 2024",Por: Joaquín Sierpe,COLUMNAS DE OPINIÓN,https://www.pivotes.cl/web/wp-content/uploads/...,2024-11-29 12:04:57
4,Ley de Inclusión Escolar: se requiere una refo...,https://www.pivotes.cl/contenidos/columnas-de-...,"26 noviembre, 2024",Por: Elisa Cabezón,COLUMNAS DE OPINIÓN,https://www.pivotes.cl/web/wp-content/uploads/...,2024-11-29 12:04:57
...,...,...,...,...,...,...,...
635,Derechos a la naturaleza… y a la réplica,https://www.pivotes.cl/contenidos/columnas-de-...,"21 diciembre, 2022",Por: Joaquín Barañao,COLUMNAS DE OPINIÓN,https://www.pivotes.cl/web/wp-content/uploads/...,2024-11-29 12:05:43
636,“La idea de que el Senado retrasa no es tan re...,https://www.pivotes.cl/contenidos/entrevistas/...,"21 diciembre, 2022",No especificado,ENTREVISTAS,https://www.pivotes.cl/web/wp-content/uploads/...,2024-11-29 12:05:43
637,DF MÁS: Bernardo Larraín conversa con Valeria ...,https://www.pivotes.cl/contenidos/entrevistas/...,"21 diciembre, 2022",No especificado,ENTREVISTAS,https://www.pivotes.cl/web/wp-content/uploads/...,2024-11-29 12:05:43
638,Ex-Ante: Columna de Bernardo Larraín: Retomar ...,https://www.pivotes.cl/contenidos/columnas-de-...,"21 diciembre, 2022",Por: Bernardo Larraín,COLUMNAS DE OPINIÓN,https://www.pivotes.cl/web/wp-content/uploads/...,2024-11-29 12:05:43


Evidencia

In [6]:
base_url = "https://www.pivotes.cl/evidencia/"
evid = extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=1, headless=False)
evid

Extrayendo artículos de: https://www.pivotes.cl/evidencia/


,Título,Enlace,Fecha Publicación,Autor,Contenido,URL Imagen,Fecha Consulta
0,Economía sin dinamismo: El gasto público crece...,https://www.pivotes.cl/evidencia/economia-sin-...,"11 noviembre, 2024",No especificado,None,https://www.pivotes.cl/web/wp-content/uploads/...,2024-11-29 12:10:04
1,Pérdida de protagonismo de la Educación Escolar,https://www.pivotes.cl/evidencia/perdida-de-pr...,"11 octubre, 2024",No especificado,None,https://www.pivotes.cl/web/wp-content/uploads/...,2024-11-29 12:10:04
2,Radiografía del sistema escolar en Chile: un a...,https://www.pivotes.cl/evidencia/radiografia-d...,"10 septiembre, 2024",No especificado,None,https://www.pivotes.cl/web/wp-content/uploads/...,2024-11-29 12:10:04
3,¿Cómo lo hizo Taiwán para acercar a la ciudada...,https://www.pivotes.cl/evidencia/como-lo-hizo-...,"6 septiembre, 2024",No especificado,None,https://www.pivotes.cl/web/wp-content/uploads/...,2024-11-29 12:10:04
4,Educación: Importancia de la apertura de los c...,https://www.pivotes.cl/evidencia/educacion-imp...,"23 agosto, 2024",No especificado,None,https://www.pivotes.cl/web/wp-content/uploads/...,2024-11-29 12:10:05
5,#PivotesPropone: ¿cómo ayudar a los actuales j...,https://www.pivotes.cl/evidencia/pivotespropon...,"22 julio, 2024",No especificado,None,https://www.pivotes.cl/web/wp-content/uploads/...,2024-11-29 12:10:05
6,Pensiones: problemas de diseño en el Seguro So...,https://www.pivotes.cl/evidencia/pensiones-pro...,"2 julio, 2024",No especificado,None,https://www.pivotes.cl/web/wp-content/uploads/...,2024-11-29 12:10:05
7,Propuesta de reforma a la Ley Lafkenche,https://www.pivotes.cl/evidencia/propuesta-de-...,"17 junio, 2024",No especificado,None,https://www.pivotes.cl/web/wp-content/uploads/...,2024-11-29 12:10:05
8,"¿Cómo lo hicieron: España, Portugal e Israel p...",https://www.pivotes.cl/evidencia/permisologia-...,"22 mayo, 2024",No especificado,None,https://www.pivotes.cl/web/wp-content/uploads/...,2024-11-29 12:10:05
9,Diagnóstico en pensiones y propuesta de reforma,https://www.pivotes.cl/evidencia/diagnostico-e...,"14 mayo, 2024",No especificado,None,https://www.pivotes.cl/web/wp-content/uploads/...,2024-11-29 12:10:05


## Concatenación

In [7]:
# Lista de DataFrames
dataframes = [contenidos, evid]

# Concatenar los DataFrames
df_concatenado = pd.concat(dataframes, ignore_index=True)

# Ver el DataFrame concatenado
df_concatenado

,Título,Enlace,Fecha Publicación,Autor,Contenido,URL Imagen,Fecha Consulta
0,Sistema político que hay que cuidar,https://www.pivotes.cl/contenidos/cartas-al-di...,"29 noviembre, 2024",Por: José Antonio Valenzuela,CARTAS AL DIRECTOR,None,2024-11-29 12:04:57
1,Si yo fuera Donald Trump,https://www.pivotes.cl/contenidos/columnas-de-...,"28 noviembre, 2024",Por: José Antonio Valenzuela,COLUMNAS DE OPINIÓN,https://www.pivotes.cl/web/wp-content/uploads/...,2024-11-29 12:04:57
2,Puntos de inflexión para el empleo público,https://www.pivotes.cl/contenidos/columnas-de-...,"28 noviembre, 2024",Por: Rafael Palacios,COLUMNAS DE OPINIÓN,https://www.pivotes.cl/web/wp-content/uploads/...,2024-11-29 12:04:57
3,Ley Lafkenche: No existen las balas de plata,https://www.pivotes.cl/contenidos/columnas-de-...,"28 noviembre, 2024",Por: Joaquín Sierpe,COLUMNAS DE OPINIÓN,https://www.pivotes.cl/web/wp-content/uploads/...,2024-11-29 12:04:57
4,Ley de Inclusión Escolar: se requiere una refo...,https://www.pivotes.cl/contenidos/columnas-de-...,"26 noviembre, 2024",Por: Elisa Cabezón,COLUMNAS DE OPINIÓN,https://www.pivotes.cl/web/wp-content/uploads/...,2024-11-29 12:04:57
...,...,...,...,...,...,...,...
671,Sexto retiro: ¿a quiénes les llega y cuáles se...,https://www.pivotes.cl/evidencia/sexto-retiro-...,"18 abril, 2023",No especificado,None,https://www.pivotes.cl/web/wp-content/uploads/...,2024-11-29 12:10:06
672,Aumento en expectativa de vida: ¿quién paga lo...,https://www.pivotes.cl/evidencia/aumento-en-ex...,"23 marzo, 2023",No especificado,None,https://www.pivotes.cl/web/wp-content/uploads/...,2024-11-29 12:10:06
673,Flexibilidad en la jornada laboral: ¿qué nos e...,https://www.pivotes.cl/evidencia/flexibilidad-...,"21 marzo, 2023",No especificado,None,https://www.pivotes.cl/web/wp-content/uploads/...,2024-11-29 12:10:07
674,Brecha de género en el mercado laboral: falta ...,https://www.pivotes.cl/evidencia/brecha-de-gen...,"8 marzo, 2023",No especificado,None,https://www.pivotes.cl/web/wp-content/uploads/...,2024-11-29 12:10:07


## Iteración sobre links

In [15]:
from selenium.webdriver.common.by import By

def extract_info_from_url(driver, url):
    """
    Extrae información de un artículo dado su URL.
    
    Args:
        driver (WebDriver): El controlador de Selenium para navegar por la página.
        url (str): La URL del artículo del cual extraer la información.
    
    Returns:
        dict: Un diccionario con la información del artículo extraído.
    """
    driver.get(url)
    
    # Esperar a que cargue el contenido (ajustar según la velocidad de carga)
    driver.implicitly_wait(1)
    
    # Intentar obtener la información:
    
    # Título del artículo (extraído del <h1>)
    try:
        title = driver.find_element(By.CSS_SELECTOR, "h1.ff-secondary.rt-h2--4").text.strip()
    except:
        title = "No disponible"
    
    # Nombre del autor (extraído del <p class="autor">)
    try:
        author = driver.find_element(By.CSS_SELECTOR, "p.autor").text.strip().replace("Por:", "").strip()
    except:
        author = "No disponible"
    
    # Fecha de publicación (extraída de <time class="updated">)
    try:
        publication_date = driver.find_element(By.CSS_SELECTOR, "time.updated").text.strip()
    except:
        publication_date = "No disponible"
    
    # Contenido principal del artículo (extraído del <div class="block-texto prosa">)
    try:
        content = driver.find_element(By.CSS_SELECTOR, "div.block-texto.prosa").text.strip()
    except:
        content = "No disponible"
    
    # Imagen destacada (extraído de <img>)
    try:
        image_url = driver.find_element(By.CSS_SELECTOR, "figure.pivotes-articulo-single__thumbnail img").get_attribute('src')
    except:
        image_url = "No disponible"
    
    # Medio de comunicación (extraído del <h4> que contiene el nombre del medio)
    try:
        media = driver.find_element(By.CSS_SELECTOR, "h4.ff-secondary.rh5.lh-condensed").text.strip()
    except:
        media = "No disponible"
    
    # Extraer la categoría y el título2 desde el breadcrumb
    try:
        breadcrumb = driver.find_element(By.CSS_SELECTOR, "div.yoast-breadcrumb")
        breadcrumb_links = breadcrumb.find_elements(By.TAG_NAME, "a")
        category = breadcrumb_links[2].text.strip()  # La tercera parte del breadcrumb (Cartas al director)
        title2 = breadcrumb.find_element(By.CSS_SELECTOR, "span.breadcrumb_last").text.strip()  # Última parte del breadcrumb
    except:
        category = "No disponible"
        title2 = "No disponible"
    
    # Crear el diccionario con los datos
    article_data = {
        "Título": title,
        "Autor": author,
        "Fecha de Publicación": publication_date,
        "Contenido": content,
        "Imagen": image_url,
        "Medio de Comunicación": media,
        "Categoría": category,
        "Título2": title2,
        "Enlace": url,
    }
    
    return article_data

In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
from tqdm import tqdm  # Importar tqdm para la barra de carga

def process_all_links(jg_df):
    """
    Procesa todos los enlaces de un DataFrame 'jg_df', extrayendo la información de cada artículo
    y almacenándola en un nuevo DataFrame 'publicaciones_jg'.

    Args:
        jg_df (pd.DataFrame): DataFrame que contiene una columna 'Enlace' con los enlaces de los artículos.

    Returns:
        pd.DataFrame: DataFrame con la información extraída de todos los enlaces.
    """
    # Configurar las opciones de Chrome para el modo headless
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Activar modo headless (sin interfaz gráfica)
    chrome_options.add_argument("--disable-gpu")  # Deshabilitar la aceleración por hardware (útil en headless)
    chrome_options.add_argument("--no-sandbox")  # Puede ser necesario en ciertos entornos (como contenedores)

    # Abrir el webdriver con las opciones configuradas
    driver = webdriver.Chrome(options=chrome_options)

    # Crear una lista para almacenar los datos de los artículos
    publicaciones_jg = []

    try:
        # Usar tqdm para mostrar la barra de carga mientras se procesan los enlaces
        for _, row in tqdm(jg_df.iterrows(), total=jg_df.shape[0], desc="Extrayendo datos de artículos"):
            url = row['Enlace']
            # Extraer la información del artículo
            article_data = extract_info_from_url(driver, url)
            
            if article_data:
                publicaciones_jg.append(article_data)
        
        # Crear un DataFrame con los artículos extraídos
        publicaciones_jg_df = pd.DataFrame(publicaciones_jg)
        
        return publicaciones_jg_df
    
    except Exception as e:
        print(f"Error en el proceso: {str(e)}")
        return pd.DataFrame()
    
    finally:
        driver.quit()  # Cerrar el webdriver al finalizar


In [17]:
aver = df_concatenado[:3]
averinfo = process_all_links(aver)
averinfo

Extrayendo datos de artículos: 100%|██████████| 3/3 [00:02<00:00,  1.07it/s]


,Título,Autor,Fecha de Publicación,Contenido,Imagen,Medio de Comunicación,Categoría,Título2,Enlace
0,SISTEMA POLÍTICO QUE HAY QUE CUIDAR,POR: JOSÉ ANTONIO VALENZUELA,"29 NOVIEMBRE, 2024",Señor director:\nEsta semana se presentó por p...,https://www.pivotes.cl/web/wp-content/uploads/...,LA TERCERA,Cartas al director,Sistema político que hay que cuidar,https://www.pivotes.cl/contenidos/cartas-al-di...
1,SI YO FUERA DONALD TRUMP,POR: JOSÉ ANTONIO VALENZUELA,"28 NOVIEMBRE, 2024","En una de mis primeras reuniones informativas,...",https://www.pivotes.cl/web/wp-content/uploads/...,EL MOSTRADOR,Columnas de opinión,Si yo fuera Donald Trump,https://www.pivotes.cl/contenidos/columnas-de-...
2,PUNTOS DE INFLEXIÓN PARA EL EMPLEO PÚBLICO,POR: RAFAEL PALACIOS,"28 NOVIEMBRE, 2024",Con el reciente dictamen de la nueva contralor...,https://www.pivotes.cl/web/wp-content/uploads/...,LA TERCERA,Columnas de opinión,Puntos de inflexión para el empleo público,https://www.pivotes.cl/contenidos/columnas-de-...


In [18]:
df_info = process_all_links(df_concatenado)
df_info

Extrayendo datos de artículos: 100%|██████████| 676/676 [33:01<00:00,  2.93s/it]


,Título,Autor,Fecha de Publicación,Contenido,Imagen,Medio de Comunicación,Categoría,Título2,Enlace
0,SISTEMA POLÍTICO QUE HAY QUE CUIDAR,POR: JOSÉ ANTONIO VALENZUELA,"29 NOVIEMBRE, 2024",Señor director:\nEsta semana se presentó por p...,https://www.pivotes.cl/web/wp-content/uploads/...,LA TERCERA,Cartas al director,Sistema político que hay que cuidar,https://www.pivotes.cl/contenidos/cartas-al-di...
1,SI YO FUERA DONALD TRUMP,POR: JOSÉ ANTONIO VALENZUELA,"28 NOVIEMBRE, 2024","En una de mis primeras reuniones informativas,...",https://www.pivotes.cl/web/wp-content/uploads/...,EL MOSTRADOR,Columnas de opinión,Si yo fuera Donald Trump,https://www.pivotes.cl/contenidos/columnas-de-...
2,PUNTOS DE INFLEXIÓN PARA EL EMPLEO PÚBLICO,POR: RAFAEL PALACIOS,"28 NOVIEMBRE, 2024",Con el reciente dictamen de la nueva contralor...,https://www.pivotes.cl/web/wp-content/uploads/...,LA TERCERA,Columnas de opinión,Puntos de inflexión para el empleo público,https://www.pivotes.cl/contenidos/columnas-de-...
3,LEY LAFKENCHE:,POR: JOAQUÍN SIERPE,"28 NOVIEMBRE, 2024","La semana pasada, el Senado despachó el Presup...",https://www.pivotes.cl/web/wp-content/uploads/...,EX-ANTE,Columnas de opinión,Ley Lafkenche: No existen las balas de plata,https://www.pivotes.cl/contenidos/columnas-de-...
4,LEY DE INCLUSIÓN ESCOLAR:,POR: ELISA CABEZÓN,"26 NOVIEMBRE, 2024",Mientras la atención del debate en torno a la ...,https://www.pivotes.cl/web/wp-content/uploads/...,CIPER CHILE,Columnas de opinión,Ley de Inclusión Escolar: se requiere una refo...,https://www.pivotes.cl/contenidos/columnas-de-...
...,...,...,...,...,...,...,...,...,...
671,SEXTO RETIRO: ¿A QUIÉNES LES LLEGA Y CUÁLES SE...,No disponible,"18 ABRIL, 2023",Algunos parlamentarios han anunciado que impul...,https://www.pivotes.cl/web/wp-content/uploads/...,ESTUDIO,No disponible,No disponible,https://www.pivotes.cl/evidencia/sexto-retiro-...
672,AUMENTO EN EXPECTATIVA DE VIDA:,No disponible,"23 MARZO, 2023","En marzo, el Senado francés aprobó aumentar ...",https://www.pivotes.cl/web/wp-content/uploads/...,ESTUDIO,No disponible,No disponible,https://www.pivotes.cl/evidencia/aumento-en-ex...
673,FLEXIBILIDAD EN LA JORNADA LABORAL: ¿QUÉ NOS E...,No disponible,"21 MARZO, 2023",En el marco del proyecto de ley que busca redu...,https://www.pivotes.cl/web/wp-content/uploads/...,ESTUDIO,No disponible,No disponible,https://www.pivotes.cl/evidencia/flexibilidad-...
674,BRECHA DE GÉNERO EN EL MERCADO LABORAL: FALTA ...,No disponible,"8 MARZO, 2023",El acceso de la mujer al mundo laboral trae mu...,https://www.pivotes.cl/web/wp-content/uploads/...,ESTUDIO,No disponible,No disponible,https://www.pivotes.cl/evidencia/brecha-de-gen...


In [19]:
df_final = df_info.copy()

In [20]:
df_final.columns

Index(['Título', 'Autor', 'Fecha de Publicación', 'Contenido', 'Imagen',
       'Medio de Comunicación', 'Categoría', 'Título2', 'Enlace'],
      dtype='object')

In [21]:
#df_final["Medio de comunicación"] = ""
df_final["Think tank"] = "Pivotes"

df_final  = df_final[['Título', 'Título2', 'Autor', 'Fecha de Publicación', 'Contenido', 'Imagen',
       'Medio de Comunicación', 'Categoría', 'Enlace']]

In [22]:
df_final.to_excel("Pivotes.xlsx", index = False)

Segunda pasada con nueva función

In [13]:
ip_elm2 = ip_elm[:2]
ip_lyr2 = ip_lyr[:2]
ip_notins2 = ip_notins[:2]

In [14]:
ip_elm_contenido = process_all_links(ip_elm)
ip_elm_contenido

Extrayendo datos de: https://ideapais.cl/noticias/columna-a-donde-estan-que-no-se-ven-por-cristian-stewart-la-segunda/
Extrayendo datos de: https://ideapais.cl/noticias/carta-al-director-el-parche-de-la-confianza-por-arnau-sarra-la-tercera/
Extrayendo datos de: https://ideapais.cl/noticias/columna-los-costos-de-mantener-a-toha-por-cristian-stewart-la-segunda/
Extrayendo datos de: https://ideapais.cl/noticias/columna-volver-a-poner-a-las-personas-en-el-centro-por-matias-domeyko-diario-el-llanquihue/
Extrayendo datos de: https://ideapais.cl/noticias/columna-solidaridad-uc-por-cristian-stewart-la-segunda/
Extrayendo datos de: https://ideapais.cl/noticias/columna-los-catolicos-y-la-conciencia-por-cristian-stewart-la-segunda/
Extrayendo datos de: https://ideapais.cl/noticias/columna-cae-victimas-y-opresores-por-magdalena-vergara-diario-la-tercera/
Extrayendo datos de: https://ideapais.cl/noticias/columna-es-justo-por-juan-pablo-lira-el-dinamo/
Extrayendo datos de: https://ideapais.cl/notici

,Título,Autor,Fecha de Publicación,Categorías,Contenido,Enlace
0,"Columna «A dónde están, que no se ven», por Cr...",IdeaPais,"Nov 21, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...",Dentro de las varias aristas que tiene el caso...,https://ideapais.cl/noticias/columna-a-donde-e...
1,"Carta al Director «El parche de la confianza»,...",María José Aravena,"Nov 14, 2024","María José Aravena, Noticias, Noticias Destaca...","Señor Director:\nEn su columna, Miriam Henríqu...",https://ideapais.cl/noticias/carta-al-director...
2,"Columna «Los costos de mantener a Tohá», por C...",IdeaPais,"Nov 7, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...",A nadie le gusta que le digan cómo uno debe ha...,https://ideapais.cl/noticias/columna-los-costo...
3,Columna «Volver a poner a las personas en el c...,IdeaPais,"Oct 25, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...","Hace algunos días, en IdeaPaís organizamos un ...",https://ideapais.cl/noticias/columna-volver-a-...
4,"Columna «Solidaridad UC», por Cristián Stewart...",IdeaPais,"Oct 24, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...","Hace 14 años, nació un proyecto político en la...",https://ideapais.cl/noticias/columna-solidarid...
5,"Columna «Los católicos y la conciencia», por C...",IdeaPais,"Sep 26, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...",El profesor Carlos Peña inició una importante ...,https://ideapais.cl/noticias/columna-los-catol...
6,"Columna «CAE: víctimas y opresores», por Magda...",IdeaPais,"Sep 18, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...",La manera en que el Frente Amplio ha enarbolad...,https://ideapais.cl/noticias/columna-cae-victi...
7,"Columna «¿Es justo?», por Juan Pablo Lira | El...",IdeaPais,"Sep 17, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...",Hablemos de la asignación de recursos en educa...,https://ideapais.cl/noticias/columna-es-justo-...
8,"Columna «Conmemoración de hechos importantes»,...",IdeaPais,"Sep 16, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...",Sin duda que las primeras semanas de septiembr...,https://ideapais.cl/noticias/columna-conmemora...


In [15]:
ip_notins_contenido = process_all_links(ip_notins)
ip_notins_contenido

Extrayendo datos de: https://ideapais.cl/noticias-institucionales/ideapais-organiza-conversatorio-sobre-desarrollo-urbano-con-alcaldes-de-puerto-montt-y-puerto-varas/
Extrayendo datos de: https://ideapais.cl/noticias-institucionales/candidatos-a-gobernador-regional-de-la-region-de-los-lagos-debatieron-en-el-evento-organizado-por-ideapais/
Extrayendo datos de: https://ideapais.cl/noticias-institucionales/ideapais-invita-a-postular-al-ii-congreso-de-pensamiento-politicas-publicas-profamilia/
Extrayendo datos de: https://ideapais.cl/noticias-institucionales/ideapais-inaugura-sede-regional-en-los-lagos-es-la-cuarta-a-nivel-nacional/
Extrayendo datos de: https://ideapais.cl/sin-categoria/ideapais-realiza-debates-entre-candidatos-a-alcaldes-en-santa-cruz-y-marchigue/
Extrayendo datos de: https://ideapais.cl/noticias-institucionales/con-presencia-de-evelyn-matthei-el-centro-de-estudios-ideapais-realizo-el-congreso-de-politicas-publicas-profamilia/
Extrayendo datos de: https://ideapais.cl/noti

,Título,Autor,Fecha de Publicación,Categorías,Contenido,Enlace
0,IdeaPaís organiza conversatorio sobre desarrol...,María José Aravena,"Nov 28, 2024","María José Aravena, Noticias Destacadas Inicio...",Rodrigo Wainraihgt y Tomás Garate dialogaron j...,https://ideapais.cl/noticias-institucionales/i...
1,Candidatos a Gobernador Regional de la región ...,María José Aravena,"Oct 15, 2024","María José Aravena, Noticias Destacadas Inicio...",El debate se enfocó en temas de seguridad y co...,https://ideapais.cl/noticias-institucionales/c...
2,IdeaPaís invita a postular al II Congreso de P...,IdeaPais,"Jun 7, 2024","IdeaPais, Noticias Institucionales, 0 Comentarios",La instancia se realizará el 27 de septiembre ...,https://ideapais.cl/noticias-institucionales/i...
3,IdeaPaís inaugura sede regional en Los Lagos: ...,IdeaPais,"May 30, 2024","IdeaPais, Noticias IdeaPaís Los Lagos, Noticia...","En el evento, su director regional, Matías Dom...",https://ideapais.cl/noticias-institucionales/i...
4,IdeaPaís realiza debates entre candidatos a al...,María José Aravena,"Oct 17, 2024","María José Aravena, Noticias Destacadas Inicio...",El centro de estudios de inspiración social cr...,https://ideapais.cl/sin-categoria/ideapais-rea...
5,Con presencia de Evelyn Matthei el centro de e...,IdeaPais,"Sep 30, 2024","IdeaPais, Noticias Destacadas Inicio, Noticias...",En el ex Congreso Nacional expusieron ex autor...,https://ideapais.cl/noticias-institucionales/c...
6,La versión 2024 del Encuentro de Líderes Jóven...,IdeaPais,"Jun 6, 2024","IdeaPais, Noticias Institucionales, Noticias O...","IdeaPaís, centro de estudios de inspiración so...",https://ideapais.cl/noticias-ohiggins/la-versi...
7,IdeaPaís y centros de estudios presentaron la ...,IdeaPais,"May 27, 2024","IdeaPais, Noticias Institucionales, 0 Comentarios",Con la presencia de la alcaldesa de Providenci...,https://ideapais.cl/noticias-institucionales/i...
8,Ex autoridades del Biobío entregan sus reflexi...,María José Aravena,"Oct 16, 2024","María José Aravena, Noticias Destacadas Inicio...",El panel de conversación se enfocó en los prin...,https://ideapais.cl/noticias-institucionales/e...
9,«En el Foco» de IdeaPaís analizó las eleccione...,IdeaPais,"Jul 17, 2024","IdeaPais, Noticias Destacadas Inicio, Noticias...",IdeaPaís llevó a cabo el evento online «Europa...,https://ideapais.cl/noticias-institucionales/e...


Crear otra funcion

In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def extract_info_from_url(driver, url):
    """
    Extrae información de un artículo dado su URL.
    
    Args:
        driver (WebDriver): El controlador de Selenium para navegar por la página.
        url (str): La URL del artículo del cual extraer la información.
    
    Returns:
        dict: Un diccionario con la información del artículo extraído.
    """
    driver.get(url)
    
    # Esperar a que cargue el contenido (se puede ajustar el tiempo de espera si es necesario)
    driver.implicitly_wait(3)  # Esperar hasta 3 segundos para que se cargue la página
    
    # Extraer la información de interés:
    try:
        # Extraer el título
        title = driver.find_element(By.CSS_SELECTOR, ".et_pb_text_inner h1").text.strip()
    except:
        title = "No disponible"
    
    try:
        # Extraer el año y número de forma específica (de la clase correcta)
        year_issue = driver.find_element(By.CSS_SELECTOR, ".et_pb_text_0 .et_pb_text_inner p").text.strip()
    except:
        year_issue = "No disponible"
    
    try:
        # Extraer todo el contenido del artículo, evitando que incluya el "Año y Número"
        # Asegurarse de que el selector no esté extrayendo elementos no deseados.
        paragraphs = driver.find_elements(By.CSS_SELECTOR, ".et_pb_text_2 .et_pb_text_inner p")
        content = "\n".join([p.text.strip() for p in paragraphs if p.text.strip() != ""])  # Unir todos los párrafos
    except:
        content = "No disponible"
    
    try:
        # Esperar hasta que el enlace de descarga esté presente
        download_link_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//a[text()='Descarga la revista aquí']"))
        )
        download_link = download_link_element.get_attribute("href")
    except Exception as e:
        print(f"Error al obtener el enlace de descarga: {e}")
        download_link = "No disponible"
    
    # Crear el diccionario con los datos
    article_data = {
        "Título": title,
        "Año y Número": year_issue,
        "Contenido": content,
        "Enlace de Descarga": download_link,
        "Enlace": url,
    }
    
    return article_data

def process_all_links(jg_df, headless=True):
    """
    Procesa todos los enlaces de un DataFrame 'jg_df', extrayendo la información de cada artículo
    y almacenándola en un nuevo DataFrame 'publicaciones_jg'.
    
    Args:
        jg_df (pd.DataFrame): DataFrame que contiene una columna 'Enlace' con los enlaces de los artículos.
        headless (bool): Determina si el modo headless debe estar activado o no. Por defecto es True.
    
    Returns:
        pd.DataFrame: DataFrame con la información extraída de todos los enlaces.
    """
    # Configurar las opciones de Chrome para el modo headless
    chrome_options = Options()
    
    if headless:
        chrome_options.add_argument("--headless")  # Activar modo headless (sin interfaz gráfica)
    
    chrome_options.add_argument("--disable-gpu")  # Deshabilitar la aceleración por hardware (útil en headless)
    chrome_options.add_argument("--no-sandbox")  # Puede ser necesario en ciertos entornos (como contenedores)

    # Abrir el webdriver con las opciones configuradas
    driver = webdriver.Chrome(options=chrome_options)

    # Crear una lista para almacenar los datos de los artículos
    publicaciones_jg = []

    try:
        for _, row in jg_df.iterrows():
            url = row['Enlace']
            print(f"Extrayendo datos de: {url}")
            
            # Extraer la información del artículo
            article_data = extract_info_from_url(driver, url)
            
            if article_data:
                publicaciones_jg.append(article_data)
        
        # Crear un DataFrame con los artículos extraídos
        publicaciones_jg_df = pd.DataFrame(publicaciones_jg)
        
        return publicaciones_jg_df
    
    except Exception as e:
        print(f"Error en el proceso: {str(e)}")
        return pd.DataFrame()
    
    finally:
        driver.quit()  # Cerrar el webdriver al finalizar

In [17]:
ip_lyr_contenido = process_all_links(ip_lyr, headless=True)
ip_lyr_contenido

Extrayendo datos de: https://ideapais.cl/revista-raices-la-familia-un-bien-publico-vi-edicion/
Extrayendo datos de: https://ideapais.cl/revista-raices-dilemas-de-la-juventud-v-edicion/
Extrayendo datos de: https://ideapais.cl/revista-raices-pensar-una-centro-derecha-reformista-iv-edicion/
Extrayendo datos de: https://ideapais.cl/revista-raices-reformismo-y-constitucion-iii-edicion/
Extrayendo datos de: https://ideapais.cl/revista-raices-desafios-constitucionales-ii-edicion/
Extrayendo datos de: https://ideapais.cl/revista-raices-solidaridad-i-edicion/
Extrayendo datos de: https://ideapais.cl/producto/gonzalo-vial-politica-y-crisis-social-2da-edicion-ampliada/
Error al obtener el enlace de descarga: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6A7556CB5+28821]
	(No symbol) [0x00007FF6A74C3840]
	(No symbol) [0x00007FF6A736578A]
	(No symbol) [0x00007FF6A73B91BE]
	(No symbol) [0x00007FF6A73B94AC]
	(No symbol) [0x00007FF6A7402647]
	(No symbol) [0x00007FF6A73DF33F]
	(No symbol) [0x0000

,Título,Año y Número,Contenido,Enlace de Descarga,Enlace
0,Revista Raíces «La familia: un bien público»,"AÑO 2024, Nº6",El motor del trabajo de IdeaPaís durante estos...,https://ideapais.cl/wp-content/uploads/2024/04...,https://ideapais.cl/revista-raices-la-familia-...
1,Revista Raíces «Dilemas de la juventud»,"AÑO 2023, Nº5",La quinta edición de Raíces está dedicada a re...,https://ideapais.cl/wp-content/uploads/2023/08...,https://ideapais.cl/revista-raices-dilemas-de-...
2,Revista Raíces «Pensar una centro derecha refo...,"AÑO 2022, Nº4","Este número de la revista Raíces, busca dar un...",https://ideapais.cl/wp-content/uploads/2023/02...,https://ideapais.cl/revista-raices-pensar-una-...
3,Revista Raíces «Reformismo y Constitución»,"AÑO 2022, Nº3",La tercera edición de la revista Raíces nos en...,https://ideapais.cl/wp-content/uploads/2022/08...,https://ideapais.cl/revista-raices-reformismo-...
4,Revista Raíces «Desafíos constitucionales»,"AÑO 2021, Nº2",La experiencia de los últimos años nos ha ense...,https://ideapais.cl/wp-content/uploads/2022/08...,https://ideapais.cl/revista-raices-desafios-co...
5,Revista Raíces «Solidaridad»,"AÑO 2020, Nº1","A diez años del nacimiento de IdeaPaís, presen...",https://ideapais.cl/wp-content/uploads/2022/08...,https://ideapais.cl/revista-raices-solidaridad...
6,No disponible,No disponible,,No disponible,https://ideapais.cl/producto/gonzalo-vial-poli...
7,No disponible,No disponible,,No disponible,https://ideapais.cl/producto/cuestiones-obreras/
8,No disponible,No disponible,,No disponible,https://ideapais.cl/producto/constitucion-soli...
9,No disponible,No disponible,,No disponible,https://ideapais.cl/producto/por-que-ha-fracas...


## Consolidar archivos

In [18]:
import pandas as pd

# Lista de DataFrames
dataframes = [ip_pp, ip_ml, ip_dc, ip_elm_contenido, ip_lyr_contenido, ip_notins_contenido, ip_inic]

# Concatenar los DataFrames
df_concatenado = pd.concat(dataframes, ignore_index=True)

# Ver el DataFrame concatenado
df_concatenado

,Título,Enlace,Fecha Publicación,URL Imagen,Contenido,Fecha Consulta,Autor,Fecha de Publicación,Categorías,Año y Número,Enlace de Descarga
0,Reporte II Congreso de Políticas Públicas Prof...,https://ideapais.cl/wp-content/uploads/2024/10...,"1 de Oct, 2024",https://ideapais.cl/wp-content/uploads/2024/10...,,2024-11-29 01:29:43,NaN,NaN,NaN,NaN,NaN
1,Presentación: Labores de cuidados en la encues...,https://ideapais.cl/wp-content/uploads/2024/09...,"3 de Sep, 2024",https://ideapais.cl/wp-content/uploads/2024/09...,,2024-11-29 01:29:43,NaN,NaN,NaN,NaN,NaN
2,Chile desde las comunas: Propuestas locales pa...,https://ideapais.cl/wp-content/uploads/2024/08...,"28 de Ago, 2024",https://ideapais.cl/wp-content/uploads/2024/08...,,2024-11-29 01:29:43,NaN,NaN,NaN,NaN,NaN
3,Radiografía: Zoom a la clase media chilena,https://ideapais.cl/wp-content/uploads/2024/07...,"18 de Jul, 2024",https://ideapais.cl/wp-content/uploads/2024/07...,,2024-11-29 01:29:43,NaN,NaN,NaN,NaN,NaN
4,Radiografía: Matrícula extranjera en el sistem...,https://ideapais.cl/wp-content/uploads/2024/07...,"5 de Jul, 2024",https://ideapais.cl/wp-content/uploads/2024/07...,,2024-11-29 01:29:43,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
97,Memoria 2020,https://ideapais.cl/wp-content/uploads/2022/08...,"Ago 12, 2022","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,2024-11-29 01:31:33,NaN,NaN,NaN,NaN,NaN
98,Memoria 2022,https://indd.adobe.com/view/2cb08e30-369b-439e...,"Feb 16, 2024","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,2024-11-29 01:31:33,NaN,NaN,NaN,NaN,NaN
99,Memoria 2019,https://ideapais.cl/wp-content/uploads/2022/08...,"Ago 12, 2022","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,2024-11-29 01:31:33,NaN,NaN,NaN,NaN,NaN
100,Memoria 2021,https://ideapais.cl/wp-content/uploads/2022/08...,"Ago 12, 2022","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,2024-11-29 01:31:33,NaN,NaN,NaN,NaN,NaN


In [19]:
df_final = df_concatenado.copy()

In [21]:
#df_final["Medio de comunicación"] = ""
df_final["Think tank"] = "Idea País"

df_final  = df_final[['Título', 'Enlace', 'Fecha Publicación', 'URL Imagen', 'Contenido', 'Autor', 'Fecha de Publicación', 'Categorías',
       'Año y Número', 'Enlace de Descarga']]

## Guardar archivos

In [22]:
df_final.to_excel("Idea País.xlsx", index = False)